In [2]:
import re
import json
import pandas as pd
from pathlib import Path

def parse_quant_params(filename):
    """
    从文件名解析量化参数
    规则：
    - 文件名开头连续数字部分长度 >=3:
      长度3: 301 → q_total=3, q_int=0
      长度4: 1001 → q_total=10, q_int=0
      长度5: 19121 → q_total=19, q_int=12
    - 其他情况视为无效格式
    """
    stem = Path(filename).stem
    num_part = re.match(r"^(\d+)", stem).group(1) if re.match(r"^\d+", stem) else ""
    
    q_total = 0
    q_int = 0
    
    # 参数解析逻辑
    if len(num_part) >= 3:
        # 总位宽解析
        if len(num_part) == 3:
            q_total = int(num_part[0])
            q_int = int(num_part[1])
        elif len(num_part) == 4:
            q_total = int(num_part[:2])
            q_int = int(num_part[2])
        elif len(num_part) >= 5:  # 处理5位及以上的情况
            q_total = int(num_part[:2])
            q_int = int(num_part[2:4])
    else:
        raise ValueError(f"文件名数字部分不足3位: {filename}")
    
    return q_total, q_int

def parse_filename(filename):
    """从文件名解析所有参数"""
    stem = Path(filename).stem
    
    # 初始化默认值
    quant_total_bits = 0
    quant_int_bits = 0
    initial_step = 0.0
    final_step = 0.0

    # 解析量化参数
    quant_pattern = r"^(\d+?)(?=_|$)"  # 匹配开头的数字部分
    quant_match = re.match(quant_pattern, stem)
    if quant_match:
        quant_total_bits, quant_int_bits = parse_quant_params(filename)
    
    # 解析剪枝参数
    prune_pattern = r"_i(\d{1,3})|_f(\d{1,3})"
    prune_matches = re.finditer(prune_pattern, stem)
    
    for match in prune_matches:
        if match.group(1):  # initial_step
            i_val = match.group(1).lstrip('0') or '0'
            initial_step = round(float(i_val)/100, 2)
        elif match.group(2):  # final_step
            f_val = match.group(2).lstrip('0') or '0'
            final_step = round(float(f_val)/100, 2)

    return quant_total_bits, quant_int_bits, initial_step, final_step

def parse_python_acc(content):
    """解析Python模型准确率"""
    first_line = content.split('\n')[0]
    acc_match = re.search(r"python模型准确率:\s*([\d.]+)%", first_line)
    return float(acc_match.group(1)) if acc_match else 0.0

def parse_file(file_path):
    """主解析函数"""
    try:
        with open(file_path, 'r', encoding='utf-8') as f:
            content = f.read()
        
        # 自动解析参数
        q_total, q_int, prune_init, prune_final = parse_filename(file_path)
        python_acc = parse_python_acc(content)
        
        # 解析精度映射表
        acc_pattern = r"fixed<(\d+),(\d+)>: ([\d.]+)%"
        accuracy_map = {(w, i): acc for w, i, acc in re.findall(acc_pattern, content)}
        
        # 解析硬件配置
        pattern = r"文件路径: fixed{(\d+)_(\d+)}.*?复用系数: (\d+).*?资源占用: ({.*?}).*?模型延迟: ([\d.]+)\s*([mu]?s)"
        matches = re.findall(pattern, content, re.DOTALL)
        
        rows = []
        for match in matches:
            hls_total, hls_int, reuse_factor, resources_str, latency_val, latency_unit = match
            
            # 处理延迟单位
            try:
                latency = float(latency_val) * 8  # Vitis综合以200MHz计算延迟
                if latency_unit.lower() == 'ms':
                    latency *= 1000
                elif latency_unit.lower() == 's':
                    latency *= 1e6
            except ValueError:
                continue
                
            # 处理资源字段
            try:
                resources = json.loads(resources_str.replace("'", "\""))
                resources = {k: int(float(v)) for k, v in resources.items()}
            except:
                resources = {'BRAM_18K': 0, 'DSP': 0, 'FF': 0, 'LUT': 0}
                
            # 构建记录
            rows.append({
                "量化总位宽": q_total,
                "量化整数位宽": q_int,
                "hls4ml总位宽": int(hls_total),
                "hls4ml整数位宽": int(hls_int),
                "Reuse Factor": int(reuse_factor),
                "剪枝inital step": prune_init,
                "剪枝final step": prune_final,
                "python模型准确率": python_acc / 100,  # 存储为小数
                "hls4ml模型准确率": float(accuracy_map.get((hls_total, hls_int), 0)) / 100,
                "BRAM占用率": resources.get("BRAM_18K", 0),
                "DSP占用率": resources.get("DSP", 0),
                "FF占用率": resources.get("FF", 0),
                "LUT占用率": resources.get("LUT", 0),
                "延迟(us)": round(latency, 3)
            })
            
        return pd.DataFrame(rows) if not pd.DataFrame(rows).empty else None
    
    except Exception as e:
        print(f"解析错误: {str(e)}")
        return None

def append_to_excel(new_df, filename):
    """改进后的数据追加函数"""
    try:
        col_formats = {
            '量化总位宽': '0',
            '量化整数位宽': '0',
            'hls4ml总位宽': '0',
            'hls4ml整数位宽': '0',
            'Reuse Factor': '0',
            'BRAM占用率': '0',
            'DSP占用率': '0',
            'FF占用率': '0',
            'LUT占用率': '0',
            '延迟(us)': '0.000',
            'python模型准确率': '0.00%',
            'hls4ml模型准确率': '0.00%',
            '剪枝inital step': '0.00',
            '剪枝final step': '0.00'
        }

        # 检查文件是否存在
        file_exists = Path(filename).exists()
        
        # 如果文件存在，直接报错
        if file_exists:
            raise FileExistsError(f"文件 {filename} 已存在，请手动删除后再运行")

        # 始终新建文件（因为不允许追加）
        with pd.ExcelWriter(filename, engine='openpyxl') as writer:
            # 写入数据并包含表头
            new_df.to_excel(writer, sheet_name='Sheet1', index=False)
            
            # 应用格式
            sheet = writer.sheets['Sheet1']
            for col, fmt in col_formats.items():
                col_idx = new_df.columns.get_loc(col)
                for row in sheet.iter_rows(min_row=2, max_row=sheet.max_row, 
                                        min_col=col_idx+1, max_col=col_idx+1):
                    for cell in row:
                        cell.number_format = fmt
        
        print(f"数据已保存到 {filename}")
        return True
    
    except FileExistsError as fee:
        print(f"错误：{str(fee)}")
        return False
    except Exception as e:
        print(f"保存失败: {str(e)}")
        return False

def find_yaml_files(directory):
    """查找目录及其子目录下所有YAML文件"""
    yaml_files = []
    path = Path(directory)
    
    if not path.exists():
        raise FileNotFoundError(f"目录 {directory} 不存在")
    
    # 递归查找所有.yaml和.yml文件
    for file_path in path.rglob("*.yaml"):
        yaml_files.append(file_path)
    for file_path in path.rglob("*.yml"):
        yaml_files.append(file_path)
    
    return sorted(yaml_files)

def process_folder(input_dir, output_file):
    """改进后的文件夹处理函数"""
    try:
        yaml_files = find_yaml_files(input_dir)
        if not yaml_files:
            print(f"在 {input_dir} 中未找到YAML文件")
            return

        print(f"找到 {len(yaml_files)} 个YAML文件, 开始处理...")
        
        # 收集所有数据
        all_data = []
        for i, file_path in enumerate(yaml_files, 1):
            try:
                print(f"正在处理文件 ({i}/{len(yaml_files)}): {file_path.name}")
                df = parse_file(file_path)
                if df is not None:
                    all_data.append(df)
                    print(f"√ 成功处理：{file_path.name}")
                else:
                    print(f"× 跳过无效文件：{file_path.name}")
            except Exception as e:
                print(f"× 处理失败：{file_path.name}，错误：{str(e)}")

        if not all_data:
            print("没有有效数据需要保存")
            return

        # 合并所有数据
        combined_df = pd.concat(all_data, ignore_index=True)
        
        # 一次性保存所有数据
        success = append_to_excel(combined_df, output_file)
        if success:
            print(f"\n处理完成! 成功处理 {len(all_data)}/{len(yaml_files)} 个文件")
        else:
            print("保存过程中发生错误")

    except Exception as e:
        print(f"处理失败: {str(e)}")

In [3]:
# 配置参数
INPUT_DIR   = "results"        # YAML文件存放目录
OUTPUT_FILE = "hls4ml_results.xlsx"  # 输出文件名

# 解析目录下的所有yaml
process_folder(INPUT_DIR, OUTPUT_FILE)

找到 37 个YAML文件, 开始处理...
正在处理文件 (1/37): 1001.yaml
√ 成功处理：1001.yaml
正在处理文件 (2/37): 1001_i000_f020.yaml
√ 成功处理：1001_i000_f020.yaml
正在处理文件 (3/37): 1001_i000_f040.yaml
√ 成功处理：1001_i000_f040.yaml
正在处理文件 (4/37): 1001_i000_f060.yaml
√ 成功处理：1001_i000_f060.yaml
正在处理文件 (5/37): 1001_i000_f080.yaml
√ 成功处理：1001_i000_f080.yaml
正在处理文件 (6/37): 1001_i000_f090.yaml
√ 成功处理：1001_i000_f090.yaml
正在处理文件 (7/37): 1101.yaml
√ 成功处理：1101.yaml
正在处理文件 (8/37): 1201.yaml
√ 成功处理：1201.yaml
正在处理文件 (9/37): 1211.yaml
√ 成功处理：1211.yaml
正在处理文件 (10/37): 1221.yaml
√ 成功处理：1221.yaml
正在处理文件 (11/37): 1231.yaml
√ 成功处理：1231.yaml
正在处理文件 (12/37): 1241.yaml
√ 成功处理：1241.yaml
正在处理文件 (13/37): 1251.yaml
√ 成功处理：1251.yaml
正在处理文件 (14/37): 1261.yaml
√ 成功处理：1261.yaml
正在处理文件 (15/37): 1301_i010_f020.yaml
√ 成功处理：1301_i010_f020.yaml
正在处理文件 (16/37): 301.yaml
√ 成功处理：301.yaml
正在处理文件 (17/37): 301_i000_f030.yaml
√ 成功处理：301_i000_f030.yaml
正在处理文件 (18/37): 301_i015_f030.yaml
√ 成功处理：301_i015_f030.yaml
正在处理文件 (19/37): 401.yaml
√ 成功处理：401.yaml
正在处理文件 (20/37): 4

In [1]:
import pandas as pd

# 读取Excel文件
df = pd.read_excel("hls4ml_results.xlsx")

# 过滤: 保留“Reuse Factor”列中值为1或者1000的行
df_filtered = df[(df['Reuse Factor'] == 1) | (df['Reuse Factor'] == 1000)]
# 将过滤后的数据保存到新的Excel文件中
df_filtered.to_excel("hls4ml_results_filtered.xlsx", index=False)

print("过滤后的文件已保存为 hls4ml_results_filtered.xlsx")

过滤后的文件已保存为 hls4ml_results_filtered.xlsx
